# Jr Transport Management - Project Architecture Overview

## Spring Boot Application Architecture Analysis

This notebook provides a comprehensive graphical architecture overview of the Jr Transport Management Spring Boot application. It demonstrates the complete flow from controllers to database mapping, including services, security, and configuration layers.

### Architecture Components:
- **Controller Layer**: REST endpoints and request handling
- **Service Layer**: Business logic and transaction management  
- **Repository Layer**: Data access and JPA operations
- **Entity Layer**: Database mapping and relationships
- **Security Layer**: JWT authentication and authorization
- **Configuration**: Database, JPA, and application settings

In [ ]:
# Import Required Libraries (Built-in only)
import os
import sys
from datetime import datetime
import json

# ASCII Art and Text-based Visualization
def print_banner(text, width=80):
    """Create a banner with text"""
    print("=" * width)
    print(f"{text:^{width}}")
    print("=" * width)

def print_box(text, width=60):
    """Create a text box"""
    lines = text.split('\n')
    print("+" + "-" * (width-2) + "+")
    for line in lines:
        print(f"| {line:<{width-4}} |")
    print("+" + "-" * (width-2) + "+")

def print_section(title, content, width=80):
    """Print a formatted section"""
    print("\n" + "=" * width)
    print(f" {title} ".center(width, "="))
    print("=" * width)
    print(content)
    print()

print("✅ Built-in libraries loaded successfully!")
print(f"📊 Ready to create text-based architecture diagrams at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("🔧 Using ASCII art and text formatting (no external dependencies required)")

: 

## 1. Project Structure Overview

This section visualizes the Spring Boot project structure showing packages, layers, and their hierarchical relationships.

In [ ]:
# Project Structure Visualization (ASCII Art)
def create_project_structure():
    structure = """
    ┌─────────────────────────────────────────────────────────────────────────┐
    │                    Jr Transport Management System                        │
    │                         (Spring Boot 3.5.3)                           │
    └─────────────────────────────────────────────────────────────────────────┘
                                        │
                    ┌───────────────────┼───────────────────┐
                    │                   │                   │
    ┌───────────────▼──────────┐ ┌──────▼──────┐ ┌─────────▼──────────┐
    │    Controller Layer      │ │ Service Layer│ │  Repository Layer  │
    │     (REST APIs)          │ │(Business Logic)│ │   (Data Access)    │
    │                          │ │              │ │                    │
    │ • AuthController         │ │ • DriverService     │ │ • DriverRepository    │
    │ • DriverController       │ │ • VehicleService    │ │ • VehicleRepository   │
    │ • VehicleController      │ │ • VehicleOwnerService│ │ • BookingRepository   │
    │ • VehicleOwnerController │ │ • UserRoleService   │ │ • CustomerRepository  │
    └──────────────────────────┘ └─────────────┘ └────────────────────┘
                    │                   │                   │
                    └───────────────────┼───────────────────┘
                                        │
    ┌───────────────────────────────────▼───────────────────────────────────┐
    │                         Entity Layer                                   │
    │                       (JPA Entities)                                   │
    │                                                                        │
    │  • Driver         • Vehicle        • VehicleOwner                     │
    │  • Booking        • Customer       • MaintenanceRecord                │
    │  • ServiceCenter  • User          • Role                              │
    └────────────────────────────────────────────────────────────────────────┘
                                        │
    ┌───────────────────────────────────▼───────────────────────────────────┐
    │                       H2 Database                                      │
    │                    (In-Memory Development)                             │
    │                                                                        │
    │  Tables: drivers, vehicles, vehicle_owners, bookings, customers        │
    │  Configuration: application-local.properties                           │
    └────────────────────────────────────────────────────────────────────────┘

    Security & Configuration Components:
    ┌──────────────────┐  ┌──────────────────┐  ┌──────────────────┐
    │  Security Config │  │   JWT Auth       │  │  CORS Policy     │
    │  • WebSecurity   │  │  • Token Gen     │  │  • Cross-Origin  │
    │  • Filter Chain  │  │  • Validation    │  │  • Local Dev     │
    └──────────────────┘  └──────────────────┘  └──────────────────┘
    """
    
    print_banner("PROJECT STRUCTURE OVERVIEW")
    print(structure)
    
    # Component Details
    print_section("LAYER RESPONSIBILITIES", """
    Controller Layer:  Handle HTTP requests, route to services, return responses
    Service Layer:     Business logic, transaction management, data validation
    Repository Layer:  Data access, JPA operations, custom queries
    Entity Layer:      Database mapping, relationships, JPA annotations
    Security Layer:    Authentication, authorization, JWT processing
    """)

create_project_structure()

## 2. Controller Layer Implementation

This section visualizes the REST controllers with their endpoints, request mappings, and HTTP methods, showing how they handle incoming requests and route them to services.

In [ ]:
# Controller Layer Architecture (ASCII Art)
def create_controller_architecture():
    controller_diagram = """
    ┌─────────────────────────────────────────────────────────────────────────────────────┐
    │                              CONTROLLER LAYER                                      │
    │                          REST API Endpoints & Mappings                             │
    └─────────────────────────────────────────────────────────────────────────────────────┘

    ┌─────────────────┐  ┌─────────────────┐  ┌─────────────────┐  ┌─────────────────┐
    │  AuthController │  │ DriverController│  │VehicleController│  │VehicleOwnerCtrl │
    │                 │  │                 │  │                 │  │                 │
    │ POST /auth/login│  │ GET /drivers    │  │ GET /vehicles   │  │ GET /owners     │
    │ POST /auth/reg  │  │ POST /drivers   │  │ POST /vehicles  │  │ POST /owners    │
    │ POST /auth/refresh│ │ PUT /drivers/{id}│ │ PUT /vehicles/{id}│ │ PUT /owners/{id}│
    │ POST /auth/logout│ │ DEL /drivers/{id}│ │ GET /vehicles/  │  │ GET /owners/    │
    │                 │  │ GET /drivers/   │  │     owner/{id}  │  │     verified    │
    │                 │  │     status/{st} │  │ GET /vehicles/  │  │ POST /owners/   │
    │                 │  │ GET /drivers/   │  │     type/{type} │  │     wallet/add  │
    │                 │  │     available   │  │                 │  │                 │
    └─────────────────┘  └─────────────────┘  └─────────────────┘  └─────────────────┘
            │                       │                       │                       │
            │                       │                       │                       │
            ▼                       ▼                       ▼                       ▼
    ┌─────────────────┐  ┌─────────────────┐  ┌─────────────────┐  ┌─────────────────┐
    │   AuthService   │  │  DriverService  │  │ VehicleService  │  │VehicleOwnerSvc  │
    │                 │  │                 │  │                 │  │                 │
    │ • authenticate  │  │ • createDriver  │  │ • saveVehicle   │  │ • saveOwner     │
    │ • generateJWT   │  │ • updateDriver  │  │ • updateStatus  │  │ • authenticate  │
    │ • validateToken │  │ • findByStatus  │  │ • findByOwner   │  │ • addToWallet   │
    │ • refreshToken  │  │ • findAvailable │  │ • findByType    │  │ • getVerified   │
    └─────────────────┘  └─────────────────┘  └─────────────────┘  └─────────────────┘
    """
    
    print_banner("CONTROLLER LAYER ARCHITECTURE")
    print(controller_diagram)
    
    http_methods = """
    HTTP METHODS USED:
    ┌─────────────────────────────────────────────────────────────────┐
    │ GET    - Retrieve data (read operations)                       │
    │ POST   - Create new resources                                   │
    │ PUT    - Update existing resources                              │
    │ DELETE - Remove resources                                       │
    └─────────────────────────────────────────────────────────────────┘
    
    ANNOTATIONS USED:
    ┌─────────────────────────────────────────────────────────────────┐
    │ @RestController  - Marks class as REST controller              │
    │ @RequestMapping  - Base URL mapping                            │
    │ @GetMapping     - GET request mapping                          │
    │ @PostMapping    - POST request mapping                         │
    │ @PutMapping     - PUT request mapping                          │
    │ @DeleteMapping  - DELETE request mapping                       │
    │ @PathVariable   - Extract URL path variables                   │
    │ @RequestBody    - Extract request body                         │
    │ @CrossOrigin    - Enable CORS for specific origins             │
    └─────────────────────────────────────────────────────────────────┘
    """
    
    print(http_methods)

create_controller_architecture()

## 3. Service Layer Design  

This section demonstrates the business logic layer showing service classes, their methods, dependency injection patterns, and service-to-service communication.

In [ ]:
# Service Layer Architecture with Business Logic
def create_service_architecture():
    fig, ax = plt.subplots(1, 1, figsize=(16, 12))
    
    # Service layer data based on actual implementation
    services = {
        'DriverService': {
            'position': (2, 9),
            'methods': [
                '@Transactional createDriver()',
                '@Transactional updateDriver()',
                'findByStatus()',
                'findAvailableDrivers()',
                'findByVerificationStatus()',
                'findByLicenseType()',
                'findByCity(), findByState()',
                'findByMinimumRating()',
                'updateDriverAccountStatus()'
            ],
            'color': '#5E81AC'
        },
        'VehicleService': {
            'position': (8, 9),
            'methods': [
                'saveVehicle()',
                'updateVehicleStatus()',
                'updateVehicleLocation()',
                'getVehiclesByOwner()',
                'getAvailableVehicles()',
                'getVehiclesWithCapacity()',
                'updateNextServiceDate()',
                'isVehicleAvailable()',
                'getVehicleStatistics()'
            ],
            'color': '#81A1C1'
        },
        'VehicleOwnerService': {
            'position': (14, 9),
            'methods': [
                '@Transactional saveVehicleOwner()',
                'authenticateOwner()',
                'getOwnersByVerificationStatus()',
                'getOwnersByAccountStatus()',
                'addToWallet()',
                'getOwnersByCity()',
                'getVerifiedOwners()',
                'getDashboardStats()'
            ],
            'color': '#88C0D0'
        }
    }
    
    # Draw service boxes with methods
    for name, data in services.items():
        x, y = data['position']
        
        # Main service box
        service_box = FancyBboxPatch(
            (x-1.5, y-0.7), 4, 1.4,
            boxstyle="round,pad=0.1",
            facecolor=data['color'],
            edgecolor='white',
            linewidth=2,
            alpha=0.9
        )
        ax.add_patch(service_box)
        
        # Service name with @Service annotation
        ax.text(x+0.5, y+0.4, f'@Service\n{name}', ha='center', va='center',
                fontsize=11, color='white', weight='bold')
        
        # Methods box
        methods_text = '\n'.join(data['methods'])
        ax.text(x+0.5, y-2.5, methods_text, ha='center', va='top',
                fontsize=8,
                bbox=dict(boxstyle="round,pad=0.5", facecolor='white', alpha=0.95))
    
    # Repository layer
    repositories = [
        ('DriverRepository', 2.5, 5.5),
        ('VehicleRepository', 8.5, 5.5),
        ('VehicleOwnerRepository', 14.5, 5.5)
    ]
    
    for repo_name, x, y in repositories:
        repo_box = FancyBboxPatch(
            (x-1.5, y-0.4), 3.5, 0.8,
            boxstyle="round,pad=0.1",
            facecolor='#D08770',
            edgecolor='white',
            linewidth=2,
            alpha=0.8
        )
        ax.add_patch(repo_box)
        
        ax.text(x+0.25, y, f'@Repository\n{repo_name}', ha='center', va='center',
                fontsize=10, color='white', weight='bold')
        
        # Arrow from service to repository
        ax.annotate('', xy=(x+0.25, y+0.4), xytext=(x+0.5, 8.3),
                   arrowprops=dict(arrowstyle='->', lw=2, color='#4C566A'))
    
    # Dependency Injection annotations
    annotations = [
        ('@Autowired', 1, 7.5),
        ('@Transactional', 5, 7.5),
        ('Business Logic', 9, 7.5),
        ('Data Validation', 13, 7.5)
    ]
    
    for annotation, x, y in annotations:
        ax.text(x, y, annotation, ha='center', va='center',
                fontsize=9, weight='bold',
                bbox=dict(boxstyle="round,pad=0.3", facecolor='#EBCB8B', alpha=0.8))
    
    # Cross-service communication
    # Arrow from VehicleService to DriverService (for vehicle assignment)
    ax.annotate('Vehicle Assignment', xy=(5, 9), xytext=(8, 9),
               arrowprops=dict(arrowstyle='<->', lw=2, color='#BF616A'),
               fontsize=9, ha='center')
    
    # Arrow from VehicleOwnerService to VehicleService (for owner's vehicles)
    ax.annotate('Owner\'s Vehicles', xy=(11, 9), xytext=(14, 9),
               arrowprops=dict(arrowstyle='<->', lw=2, color='#BF616A'),
               fontsize=9, ha='center')
    
    # Transaction management box
    tx_box = FancyBboxPatch(
        (17, 8), 3, 2,
        boxstyle="round,pad=0.2",
        facecolor='#A3BE8C',
        edgecolor='white',
        linewidth=2,
        alpha=0.8
    )
    ax.add_patch(tx_box)
    
    ax.text(18.5, 9, '@Transactional\nManagement\n\n• ACID Properties\n• Rollback Support\n• Read-Only Ops',
            ha='center', va='center', fontsize=9, color='white', weight='bold')
    
    ax.set_xlim(0, 21)
    ax.set_ylim(3, 11)
    ax.set_title('Service Layer - Business Logic & Transaction Management', 
                fontsize=16, weight='bold', pad=20)
    ax.axis('off')
    
    plt.tight_layout()
    plt.show()

create_service_architecture()

## 4. Repository Layer and Database Mapping

This section shows repository interfaces extending JPA repositories, CRUD operations, custom query methods, and the data access layer architecture.

In [ ]:
# Repository Layer and Data Access Architecture
def create_repository_architecture():
    fig, ax = plt.subplots(1, 1, figsize=(18, 12))
    
    # Repository data based on actual implementation
    repositories = {
        'DriverRepository': {
            'position': (3, 9),
            'methods': [
                '// Inherited from JpaRepository',
                'findAll(), findById(), save(), delete()',
                '// Custom Query Methods',
                'findByStatus()',
                'findByVerificationStatus()',
                'findByLicenseType()',
                'findByCity(), findByState()',
                'findByAverageRatingGreaterThanEqual()',
                '@Query findTopRatedDrivers()',
                '@Query findDriversWithExpiringLicenses()'
            ],
            'color': '#5E81AC'
        },
        'VehicleRepository': {
            'position': (9, 9),
            'methods': [
                '// Inherited from JpaRepository',
                'findAll(), findById(), save(), delete()',
                '// Custom Query Methods',
                'findByOwnerId()',
                'findByDriverId()',
                'findByVehicleNumber()',
                'findByStatus()',
                'findByVehicleType()',
                '@Query findByLocationCityAndState()',
                '@Query findVehiclesWithExpiringDocuments()'
            ],
            'color': '#81A1C1'
        },
        'BookingRepository': {
            'position': (15, 9),
            'methods': [
                '// Inherited from JpaRepository',
                'findAll(), findById(), save(), delete()',
                '// Custom Query Methods',
                'findByCustomerId()',
                'findByDriverId(), findByVehicleId()',
                'findByStatus()',
                '@Query findActiveBookings()',
                '@Query findBookingsByDateRange()',
                '@Query findByDepartureCity()',
                '@Query findOverdueBookings()'
            ],
            'color': '#88C0D0'
        }
    }
    
    # Draw repository boxes with methods
    for name, data in repositories.items():
        x, y = data['position']
        
        # Main repository box
        repo_box = FancyBboxPatch(
            (x-2, y-0.7), 5, 1.4,
            boxstyle="round,pad=0.1",
            facecolor=data['color'],
            edgecolor='white',
            linewidth=2,
            alpha=0.9
        )
        ax.add_patch(repo_box)
        
        # Repository name with interface annotation
        ax.text(x+0.5, y+0.3, f'@Repository\ninterface {name}\nextends JpaRepository<Entity, String>',
                ha='center', va='center', fontsize=10, color='white', weight='bold')
        
        # Methods box
        methods_text = '\n'.join(data['methods'])
        ax.text(x+0.5, y-3, methods_text, ha='center', va='top',
                fontsize=8,
                bbox=dict(boxstyle="round,pad=0.5", facecolor='white', alpha=0.95))
    
    # JPA Repository inheritance
    jpa_box = FancyBboxPatch(
        (8.5, 11.5), 4, 1,
        boxstyle="round,pad=0.1",
        facecolor='#D08770',
        edgecolor='white',
        linewidth=2,
        alpha=0.9
    )
    ax.add_patch(jpa_box)
    
    ax.text(10.5, 12, 'JpaRepository<T, ID>\n(Spring Data JPA)',
            ha='center', va='center', fontsize=11, color='white', weight='bold')
    
    # Inheritance arrows
    for x in [3.5, 9.5, 15.5]:
        ax.annotate('', xy=(x, 9.7), xytext=(10.5, 11.5),
                   arrowprops=dict(arrowstyle='->', lw=2, color='#4C566A', linestyle='--'))
    
    # Database layer
    db_entities = [
        ('drivers table', 3, 4.5),
        ('vehicles table', 9, 4.5),
        ('bookings table', 15, 4.5)
    ]
    
    for table_name, x, y in db_entities:
        table_box = FancyBboxPatch(
            (x-1.5, y-0.5), 3.5, 1,
            boxstyle="round,pad=0.1",
            facecolor='#8FBCBB',
            edgecolor='white',
            linewidth=2,
            alpha=0.8
        )
        ax.add_patch(table_box)
        
        ax.text(x+0.25, y, f'H2 Database\n{table_name}', ha='center', va='center',
                fontsize=10, color='white', weight='bold')
        
        # Arrow from repository to database
        ax.annotate('', xy=(x+0.25, y+0.5), xytext=(x+0.5, 8.3),
                   arrowprops=dict(arrowstyle='->', lw=2, color='#4C566A'))
    
    # Query types legend
    query_types = [
        ('Method Name Queries', '#A3BE8C', 'findByFieldName()'),
        ('@Query Annotations', '#EBCB8B', 'Custom JPQL/SQL'),
        ('Inherited Methods', '#D08770', 'CRUD Operations'),
        ('Pagination Support', '#BF616A', 'Pageable interface')
    ]
    
    for i, (query_type, color, description) in enumerate(query_types):
        legend_box = FancyBboxPatch(
            (1, 7 - i*1.2), 4, 0.8,
            boxstyle="round,pad=0.1",
            facecolor=color,
            alpha=0.8
        )
        ax.add_patch(legend_box)
        ax.text(3, 7.4 - i*1.2, f'{query_type}\n{description}', ha='center', va='center',
                fontsize=9, color='white', weight='bold')
    
    ax.text(3, 8, 'Repository Query Types', ha='center', va='center',
            fontsize=12, weight='bold')
    
    ax.set_xlim(0, 19)
    ax.set_ylim(1, 13.5)
    ax.set_title('Repository Layer - Data Access & JPA Integration', 
                fontsize=16, weight='bold', pad=20)
    ax.axis('off')
    
    plt.tight_layout()
    plt.show()

create_repository_architecture()

## 5. Entity Relationships and JPA Configuration

This section visualizes entity classes with JPA annotations, relationships (OneToMany, ManyToOne, etc.), and the database schema mapping using H2 configuration.

In [ ]:
# Entity Relationships and Database Schema
def create_entity_relationships():
    fig, ax = plt.subplots(1, 1, figsize=(18, 14))
    
    # Entity data based on actual implementation
    entities = {
        'Driver': {
            'position': (3, 11),
            'fields': [
                '@Id String id (UUID)',
                'String userId, firstName, lastName',
                'String email, phone',
                'DriverStatus status',
                'VerificationStatus verificationStatus',
                'LicenseType licenseType',
                'BigDecimal averageRating',
                'Integer totalYearsExperience',
                '@Embedded Address address',
                '@CreationTimestamp createdAt'
            ],
            'color': '#5E81AC'
        },
        'Vehicle': {
            'position': (9, 11),
            'fields': [
                '@Id String id (UUID)',
                'String vehicleNumber, ownerId, driverId',
                'VehicleType vehicleType',
                'VehicleStatus status',
                '@Column(name="`year`") Integer year',
                'String model, brand',
                'BigDecimal capacity',
                '@Embedded DocumentInfo registration',
                '@Embedded InsuranceInfo insurance',
                'Boolean isActive, isVerified'
            ],
            'color': '#81A1C1'
        },
        'VehicleOwner': {
            'position': (15, 11),
            'fields': [
                '@Id String ownerId (UUID)',
                'String firstName, lastName',
                'String email, phone, password',
                'VerificationStatus verificationStatus',
                'AccountStatus accountStatus',
                'BigDecimal walletBalance',
                '@Embedded Address address',
                '@Embedded IdentityProof identityProof',
                '@CreationTimestamp createdAt'
            ],
            'color': '#88C0D0'
        },
        'Booking': {
            'position': (3, 7),
            'fields': [
                '@Id String id (UUID)',
                'String customerId, driverId',
                'String vehicleId, ownerId',
                'BookingStatus status',
                'PaymentStatus paymentStatus',
                '@Embedded Address pickupAddress',
                '@Embedded Address deliveryAddress',
                '@Embedded PricingInfo pricing',
                'LocalDateTime scheduledPickupDate'
            ],
            'color': '#8FBCBB'
        },
        'Customer': {
            'position': (9, 7),
            'fields': [
                '@Id String customerId (UUID)',
                'String firstName, lastName',
                'String email, phone',
                'VerificationStatus verificationStatus',
                '@Embedded Address address',
                '@Embedded BankDetails bankDetails',
                'String preferredVehicleType',
                '@CreationTimestamp createdAt'
            ],
            'color': '#A3BE8C'
        },
        'MaintenanceRecord': {
            'position': (15, 7),
            'fields': [
                '@Id String id (UUID)',
                'String vehicleId',
                'String description',
                'BigDecimal cost',
                'LocalDate serviceDate',
                'MaintenanceType type',
                'String serviceCenterName',
                '@ForeignKey(NO_CONSTRAINT)'
            ],
            'color': '#EBCB8B'
        }
    }
    
    # Draw entity boxes
    for name, data in entities.items():
        x, y = data['position']
        
        # Entity box
        entity_box = FancyBboxPatch(
            (x-1.5, y-1), 4, 2,
            boxstyle="round,pad=0.1",
            facecolor=data['color'],
            edgecolor='white',
            linewidth=2,
            alpha=0.9
        )
        ax.add_patch(entity_box)
        
        # Entity name
        ax.text(x+0.5, y+0.7, f'@Entity\n@Table(name="{name.lower()}s")\n{name}',
                ha='center', va='center', fontsize=10, color='white', weight='bold')
        
        # Fields
        fields_text = '\n'.join(data['fields'][:6])  # Show first 6 fields
        ax.text(x+0.5, y-2.5, fields_text, ha='center', va='top',
                fontsize=7,
                bbox=dict(boxstyle="round,pad=0.3", facecolor='white', alpha=0.95))
    
    # Relationships
    relationships = [
        # VehicleOwner to Vehicle (One-to-Many)
        {'from': (15.5, 10), 'to': (10.5, 11), 'label': '1:N\nOwns', 'color': '#BF616A'},
        # Driver to Vehicle (Many-to-One for current assignment)
        {'from': (4.5, 11), 'to': (8, 11), 'label': 'M:1\nDrives', 'color': '#BF616A'},
        # Customer to Booking (One-to-Many)
        {'from': (9, 8), 'to': (4.5, 7), 'label': '1:N\nBooks', 'color': '#D08770'},
        # Vehicle to Booking (One-to-Many)
        {'from': (9, 10), 'to': (4.5, 8), 'label': '1:N\nUsed in', 'color': '#D08770'},
        # Vehicle to MaintenanceRecord (One-to-Many)
        {'from': (11, 10), 'to': (14, 8), 'label': '1:N\nMaintenance', 'color': '#A3BE8C'}
    ]
    
    for rel in relationships:
        ax.annotate(rel['label'], xy=rel['to'], xytext=rel['from'],
                   arrowprops=dict(arrowstyle='->', lw=2, color=rel['color']),
                   fontsize=9, ha='center', weight='bold',
                   bbox=dict(boxstyle="round,pad=0.2", facecolor='white', alpha=0.8))
    
    # JPA Annotations legend
    jpa_annotations = [
        ('@Entity', 'Marks class as JPA entity'),
        ('@Table', 'Maps to database table'),
        ('@Id', 'Primary key field'),
        ('@GeneratedValue', 'Auto-generated UUID'),
        ('@Column', 'Custom column mapping'),
        ('@Embedded', 'Embedded objects'),
        ('@CreationTimestamp', 'Auto timestamp'),
        ('@ForeignKey', 'Foreign key constraints')
    ]
    
    legend_y = 4.5
    for i, (annotation, description) in enumerate(jpa_annotations):
        if i < 4:  # First column
            x_pos, y_pos = 1, legend_y - i*0.5
        else:  # Second column
            x_pos, y_pos = 10, legend_y - (i-4)*0.5
            
        ax.text(x_pos, y_pos, f'{annotation}: {description}', 
                fontsize=8, weight='bold',
                bbox=dict(boxstyle="round,pad=0.2", facecolor='#ECEFF4', alpha=0.9))
    
    # H2 Database configuration
    h2_config = FancyBboxPatch(
        (1, 1.5), 16, 1.5,
        boxstyle="round,pad=0.2",
        facecolor='#4C566A',
        edgecolor='white',
        linewidth=2,
        alpha=0.9
    )
    ax.add_patch(h2_config)
    
    config_text = """H2 Database Configuration (application-local.properties):
spring.datasource.url=jdbc:h2:mem:testdb  |  spring.jpa.hibernate.ddl-auto=create-drop  |  spring.jpa.database-platform=org.hibernate.dialect.H2Dialect
In-memory database for development  |  Auto-create/drop schema  |  H2-specific SQL dialect"""
    
    ax.text(9, 2.25, config_text, ha='center', va='center',
            fontsize=10, color='white', weight='bold')
    
    ax.set_xlim(0, 19)
    ax.set_ylim(0.5, 13.5)
    ax.set_title('Entity Relationships & JPA Configuration - Database Schema Mapping', 
                fontsize=16, weight='bold', pad=20)
    ax.axis('off')
    
    plt.tight_layout()
    plt.show()

create_entity_relationships()

## 6. Security Layer Integration

This section demonstrates JWT-based security configuration, authentication flow, and how security integrates with the controller layer.

In [ ]:
# Security Layer and JWT Authentication Flow
def create_security_architecture():
    fig, ax = plt.subplots(1, 1, figsize=(16, 12))
    
    # Authentication flow steps
    auth_steps = [
        {'name': 'Client Request', 'pos': (2, 10), 'color': '#5E81AC'},
        {'name': 'Security Filter Chain', 'pos': (6, 10), 'color': '#81A1C1'},
        {'name': 'JWT Token Validation', 'pos': (10, 10), 'color': '#88C0D0'},
        {'name': 'Authentication Manager', 'pos': (14, 10), 'color': '#8FBCBB'},
        {'name': 'UserDetailsService', 'pos': (6, 7), 'color': '#A3BE8C'},
        {'name': 'Controller Access', 'pos': (10, 7), 'color': '#EBCB8B'}
    ]
    
    # Draw authentication flow boxes
    for step in auth_steps:
        x, y = step['pos']
        box = FancyBboxPatch(
            (x-1, y-0.5), 3, 1,
            boxstyle="round,pad=0.1",
            facecolor=step['color'],
            edgecolor='white',
            linewidth=2,
            alpha=0.9
        )
        ax.add_patch(box)
        
        ax.text(x+0.5, y, step['name'], ha='center', va='center',
                fontsize=10, color='white', weight='bold')
    
    # Authentication flow arrows
    flow_arrows = [
        ((3, 10), (5, 10)),  # Client to Security Filter
        ((7, 10), (9, 10)),  # Security Filter to JWT Validation
        ((11, 10), (13, 10)),  # JWT to Auth Manager
        ((14, 9.5), (6.5, 7.5)),  # Auth Manager to UserDetailsService
        ((7, 7), (9, 7)),  # UserDetailsService to Controller
    ]
    
    for start, end in flow_arrows:
        ax.annotate('', xy=end, xytext=start,
                   arrowprops=dict(arrowstyle='->', lw=2, color='#4C566A'))
    
    # Security Configuration components
    security_configs = [
        {'name': 'WebSecurityConfig', 'pos': (2, 5), 'details': [
            '@Configuration',
            '@EnableWebSecurity',
            'SecurityFilterChain configuration',
            'CORS configuration',
            'JWT filter registration',
            'Public endpoints: /api/auth/**'
        ]},
        {'name': 'JwtAuthenticationFilter', 'pos': (8, 5), 'details': [
            'OncePerRequestFilter',
            'Extract JWT from header',
            'Validate token signature',
            'Set SecurityContext',
            'Forward to next filter'
        ]},
        {'name': 'JwtUtil', 'pos': (14, 5), 'details': [
            'Token generation',
            'Token validation',
            'Claims extraction',
            'Expiration handling',
            'Secret key management'
        ]}
    ]
    
    for config in security_configs:
        x, y = config['pos']
        
        # Main config box
        config_box = FancyBboxPatch(
            (x-1.5, y-0.5), 4, 1,
            boxstyle="round,pad=0.1",
            facecolor='#D08770',
            edgecolor='white',
            linewidth=2,
            alpha=0.9
        )
        ax.add_patch(config_box)
        
        ax.text(x+0.5, y, config['name'], ha='center', va='center',
                fontsize=11, color='white', weight='bold')
        
        # Details box
        details_text = '\n'.join(config['details'])
        ax.text(x+0.5, y-2, details_text, ha='center', va='top',
                fontsize=8,
                bbox=dict(boxstyle="round,pad=0.4", facecolor='white', alpha=0.95))
    
    # JWT Token structure
    jwt_box = FancyBboxPatch(
        (1, 1), 14, 1.5,
        boxstyle="round,pad=0.2",
        facecolor='#BF616A',
        edgecolor='white',
        linewidth=2,
        alpha=0.9
    )
    ax.add_patch(jwt_box)
    
    jwt_text = """JWT Token Structure (from application-local.properties):
jwt.secret=myDefaultSecretKey... (Base64 encoded)  |  jwt.expiration=86400000 (24 hours)
Header: {"alg":"HS256","typ":"JWT"}  |  Payload: {"sub":"userId","iat":timestamp,"exp":timestamp}  |  Signature: HMACSHA256(base64UrlEncode(header) + "." + base64UrlEncode(payload), secret)"""
    
    ax.text(8, 1.75, jwt_text, ha='center', va='center',
            fontsize=9, color='white', weight='bold')
    
    # Security annotations
    annotations = [
        ('@PreAuthorize', 16, 8.5),
        ('@Secured', 16, 7.5),
        ('@RolesAllowed', 16, 6.5),
        ('@CrossOrigin', 16, 5.5)
    ]
    
    for annotation, x, y in annotations:
        ax.text(x, y, annotation, ha='center', va='center',
                fontsize=9, weight='bold',
                bbox=dict(boxstyle="round,pad=0.3", facecolor='#ECEFF4', alpha=0.9))
    
    ax.text(16, 9, 'Security\nAnnotations', ha='center', va='center',
            fontsize=11, weight='bold')
    
    ax.set_xlim(0, 18)
    ax.set_ylim(0, 11.5)
    ax.set_title('Security Layer - JWT Authentication & Authorization Flow', 
                fontsize=16, weight='bold', pad=20)
    ax.axis('off')
    
    plt.tight_layout()
    plt.show()

create_security_architecture()

## 7. Application Configuration Analysis

This section parses and visualizes the application properties configuration, showing database connections, JPA settings, and security configurations with their architectural impacts.

In [ ]:
# Application Configuration Analysis
def create_configuration_analysis():
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 12))
    
    # Configuration categories
    config_categories = {
        'Database Configuration': {
            'properties': [
                'spring.datasource.url=jdbc:h2:mem:testdb',
                'spring.datasource.driverClassName=org.h2.Driver',
                'spring.h2.console.enabled=true',
                'spring.jpa.database-platform=org.hibernate.dialect.H2Dialect'
            ],
            'color': '#5E81AC',
            'position': (2, 8)
        },
        'JPA/Hibernate Configuration': {
            'properties': [
                'spring.jpa.hibernate.ddl-auto=create-drop',
                'spring.jpa.show-sql=true',
                'spring.jpa.properties.hibernate.format_sql=true',
                'spring.jpa.defer-datasource-initialization=false'
            ],
            'color': '#81A1C1',
            'position': (8, 8)
        },
        'Security Configuration': {
            'properties': [
                'jwt.secret=myDefaultSecretKey...',
                'jwt.expiration=86400000',
                'cors.allowed-origins=http://localhost:3000,4200'
            ],
            'color': '#88C0D0',
            'position': (14, 8)
        },
        'Server Configuration': {
            'properties': [
                'server.port=8080',
                'spring.application.name=playschool-management-backend',
                'spring.servlet.multipart.max-file-size=10MB'
            ],
            'color': '#8FBCBB',
            'position': (2, 4)
        },
        'Logging Configuration': {
            'properties': [
                'logging.level.com.playschool.management=DEBUG',
                'logging.level.org.hibernate.SQL=DEBUG',
                'logging.level.org.springframework.web=INFO'
            ],
            'color': '#A3BE8C',
            'position': (8, 4)
        },
        'Additional Hibernate': {
            'properties': [
                'spring.jpa.properties.hibernate.order_updates=true',
                'spring.jpa.properties.hibernate.hbm2ddl.halt_on_error=false',
                'spring.jpa.properties.hibernate.hbm2ddl.create_namespaces=true'
            ],
            'color': '#EBCB8B',
            'position': (14, 4)
        }
    }
    
    # Draw configuration boxes
    for category, data in config_categories.items():
        x, y = data['position']
        
        # Category box
        cat_box = FancyBboxPatch(
            (x-1.5, y-0.5), 4.5, 1,
            boxstyle="round,pad=0.1",
            facecolor=data['color'],
            edgecolor='white',
            linewidth=2,
            alpha=0.9
        )
        ax1.add_patch(cat_box)
        
        ax1.text(x+0.75, y, category, ha='center', va='center',
                fontsize=10, color='white', weight='bold')
        
        # Properties box
        props_text = '\n'.join(data['properties'])
        ax1.text(x+0.75, y-2, props_text, ha='center', va='top',
                fontsize=7,
                bbox=dict(boxstyle="round,pad=0.4", facecolor='white', alpha=0.95))
    
    ax1.set_xlim(0, 18)
    ax1.set_ylim(1, 10)
    ax1.set_title('Application Configuration Properties (application-local.properties)', 
                 fontsize=14, weight='bold')
    ax1.axis('off')
    
    # Configuration impact flow chart
    impact_flow = [
        {'component': 'H2 Database', 'pos': (2, 7), 'impacts': [
            'In-memory storage',
            'Fast startup',
            'Development friendly',
            'No persistence'
        ]},
        {'component': 'Hibernate DDL', 'pos': (6, 7), 'impacts': [
            'Auto schema creation',
            'Table generation',
            'Foreign key handling',
            'Development lifecycle'
        ]},
        {'component': 'JWT Security', 'pos': (10, 7), 'impacts': [
            'Stateless authentication',
            'Token-based access',
            '24-hour expiration',
            'HMAC signature'
        ]},
        {'component': 'CORS Policy', 'pos': (14, 7), 'impacts': [
            'Cross-origin requests',
            'Frontend integration',
            'Local development',
            'Security headers'
        ]}
    ]
    
    for component in impact_flow:
        x, y = component['pos']
        
        # Component box
        comp_box = FancyBboxPatch(
            (x-1, y-0.4), 3, 0.8,
            boxstyle="round,pad=0.1",
            facecolor='#D08770',
            edgecolor='white',
            linewidth=2,
            alpha=0.9
        )
        ax2.add_patch(comp_box)
        
        ax2.text(x+0.5, y, component['component'], ha='center', va='center',
                fontsize=10, color='white', weight='bold')
        
        # Impact box
        impacts_text = '\n'.join(component['impacts'])
        ax2.text(x+0.5, y-2, impacts_text, ha='center', va='top',
                fontsize=8,
                bbox=dict(boxstyle="round,pad=0.3", facecolor='#ECEFF4', alpha=0.95))
        
        # Arrow to impact
        ax2.annotate('', xy=(x+0.5, y-1.2), xytext=(x+0.5, y-0.4),
                    arrowprops=dict(arrowstyle='->', lw=2, color='#4C566A'))
    
    # Central application box
    app_box = FancyBboxPatch(
        (6, 3.5), 4, 1,
        boxstyle="round,pad=0.1",
        facecolor='#BF616A',
        edgecolor='white',
        linewidth=3,
        alpha=0.9
    )
    ax2.add_patch(app_box)
    
    ax2.text(8, 4, 'Jr Transport Management\nSpring Boot Application', 
            ha='center', va='center', fontsize=11, color='white', weight='bold')
    
    # Arrows from components to application
    for component in impact_flow:
        x, y = component['pos']
        ax2.annotate('', xy=(7.5, 4.5), xytext=(x+0.5, y-0.4),
                    arrowprops=dict(arrowstyle='->', lw=2, color='#4C566A', alpha=0.7))
    
    ax2.set_xlim(0, 16)
    ax2.set_ylim(1, 8)
    ax2.set_title('Configuration Impact on Application Architecture', 
                 fontsize=14, weight='bold')
    ax2.axis('off')
    
    plt.tight_layout()
    plt.show()

create_configuration_analysis()

## 8. Complete Architecture Flow Visualization

This section creates comprehensive diagrams showing the complete request-response flow from client to database, including all layers, components, and their interactions.

In [ ]:
# Complete Architecture Flow - End-to-End Request Processing
def create_complete_architecture_flow():
    fig, ax = plt.subplots(1, 1, figsize=(20, 14))
    
    # Define layers with their components
    layers = {
        'Client Layer': {
            'y': 12,
            'components': ['React/Angular Frontend', 'Mobile App', 'Postman/API Client'],
            'color': '#2E3440'
        },
        'Load Balancer/Gateway': {
            'y': 10.5,
            'components': ['Spring Cloud Gateway', 'Nginx', 'API Gateway'],
            'color': '#3B4252'
        },
        'Security Layer': {
            'y': 9,
            'components': ['JWT Filter', 'CORS Filter', 'Authentication Manager'],
            'color': '#BF616A'
        },
        'Controller Layer': {
            'y': 7.5,
            'components': ['AuthController', 'DriverController', 'VehicleController', 'VehicleOwnerController'],
            'color': '#5E81AC'
        },
        'Service Layer': {
            'y': 6,
            'components': ['DriverService', 'VehicleService', 'VehicleOwnerService', 'UserRoleService'],
            'color': '#81A1C1'
        },
        'Repository Layer': {
            'y': 4.5,
            'components': ['DriverRepository', 'VehicleRepository', 'BookingRepository', 'CustomerRepository'],
            'color': '#88C0D0'
        },
        'Database Layer': {
            'y': 3,
            'components': ['H2 In-Memory Database', 'JPA/Hibernate', 'Connection Pool'],
            'color': '#8FBCBB'
        }
    }
    
    # Draw layers
    layer_width = 18
    for layer_name, layer_data in layers.items():
        y = layer_data['y']
        
        # Layer background
        layer_bg = FancyBboxPatch(
            (1, y-0.4), layer_width, 0.8,
            boxstyle="round,pad=0.1",
            facecolor=layer_data['color'],
            edgecolor='white',
            linewidth=2,
            alpha=0.3
        )
        ax.add_patch(layer_bg)
        
        # Layer label
        ax.text(0.5, y, layer_name, ha='right', va='center',
                fontsize=11, weight='bold', rotation=90)
        
        # Components
        component_width = layer_width / len(layer_data['components'])
        for i, component in enumerate(layer_data['components']):
            x = 1 + i * component_width + component_width/2
            
            comp_box = FancyBboxPatch(
                (x-component_width/2+0.1, y-0.3), component_width-0.2, 0.6,
                boxstyle="round,pad=0.05",
                facecolor=layer_data['color'],
                edgecolor='white',
                linewidth=1,
                alpha=0.9
            )
            ax.add_patch(comp_box)
            
            ax.text(x, y, component, ha='center', va='center',
                    fontsize=8, color='white', weight='bold')
    
    # Request flow arrows
    flow_steps = [
        (10, 12, 10, 10.5, 'HTTP Request\n(GET /api/vehicles)'),
        (10, 10.5, 10, 9, 'Security Check\n(JWT Validation)'),
        (10, 9, 10, 7.5, 'Route to Controller\n(@GetMapping)'),
        (10, 7.5, 10, 6, 'Business Logic\n(@Transactional)'),
        (10, 6, 10, 4.5, 'Data Access\n(JPA Repository)'),
        (10, 4.5, 10, 3, 'Database Query\n(SQL/JPQL)')
    ]
    
    for x1, y1, x2, y2, label in flow_steps:
        # Request arrow (blue)
        ax.annotate('', xy=(x2-0.3, y2+0.3), xytext=(x1-0.3, y1-0.3),
                   arrowprops=dict(arrowstyle='->', lw=3, color='#4C566A'))
        
        # Response arrow (green)
        ax.annotate('', xy=(x1+0.3, y1-0.3), xytext=(x2+0.3, y2+0.3),
                   arrowprops=dict(arrowstyle='->', lw=3, color='#A3BE8C'))
        
        # Label
        ax.text(x1+1.5, (y1+y2)/2, label, ha='left', va='center',
                fontsize=9, weight='bold',
                bbox=dict(boxstyle="round,pad=0.3", facecolor='white', alpha=0.9))
    
    # Response flow labels
    response_steps = [
        (12, 2.5, 'SQL Result Set'),
        (12, 4, 'Entity Objects'),
        (12, 5.25, 'Business Objects'),
        (12, 6.75, 'JSON Response'),
        (12, 8.25, 'Security Headers'),
        (12, 9.75, 'HTTP Response'),
        (12, 11.25, 'UI Update')
    ]
    
    for x, y, label in response_steps:
        ax.text(x, y, f'← {label}', ha='left', va='center',
                fontsize=8, color='#A3BE8C', weight='bold')
    
    # Cross-cutting concerns
    concerns = [
        {'name': 'Exception Handling\n@ControllerAdvice', 'pos': (16, 8.5), 'color': '#D08770'},
        {'name': 'Validation\n@Valid, @Validated', 'pos': (16, 7), 'color': '#EBCB8B'},
        {'name': 'Logging\n@Slf4j, Logback', 'pos': (16, 5.5), 'color': '#B48EAD'},
        {'name': 'Caching\n@Cacheable', 'pos': (16, 4), 'color': '#A3BE8C'}
    ]
    
    for concern in concerns:
        x, y = concern['pos']
        concern_box = FancyBboxPatch(
            (x-0.8, y-0.4), 2.6, 0.8,
            boxstyle="round,pad=0.1",
            facecolor=concern['color'],
            edgecolor='white',
            linewidth=2,
            alpha=0.8
        )
        ax.add_patch(concern_box)
        
        ax.text(x+0.5, y, concern['name'], ha='center', va='center',
                fontsize=9, color='white', weight='bold')
    
    # Technology stack
    tech_stack = FancyBboxPatch(
        (1, 0.5), 18, 1.5,
        boxstyle="round,pad=0.2",
        facecolor='#4C566A',
        edgecolor='white',
        linewidth=2,
        alpha=0.9
    )
    ax.add_patch(tech_stack)
    
    tech_text = """Technology Stack:
Spring Boot 3.5.3 | Spring Data JPA | Spring Security | JWT | H2 Database | Hibernate 6.6.18 | Maven | Lombok | Swagger/OpenAPI | CORS | Validation API"""
    
    ax.text(10, 1.25, tech_text, ha='center', va='center',
            fontsize=10, color='white', weight='bold')
    
    # Legend
    ax.text(15, 11, 'Request Flow →', ha='left', va='center',
            fontsize=10, color='#4C566A', weight='bold')
    ax.text(15, 10.5, '← Response Flow', ha='left', va='center',
            fontsize=10, color='#A3BE8C', weight='bold')
    
    ax.set_xlim(0, 20)
    ax.set_ylim(0, 13)
    ax.set_title('Jr Transport Management - Complete Architecture Flow\nRequest-Response Lifecycle Through All Layers', 
                fontsize=16, weight='bold', pad=20)
    ax.axis('off')
    
    plt.tight_layout()
    plt.show()

create_complete_architecture_flow()

## Summary & Analysis

### Key Architectural Insights:

1. **Layered Architecture**: Clean separation of concerns with Controller → Service → Repository → Entity layers
2. **Spring Boot Integration**: Leverages Spring's dependency injection and auto-configuration
3. **JPA/Hibernate**: Entity relationships with proper annotations for database mapping
4. **Security**: JWT-based stateless authentication with CORS support
5. **H2 Database**: In-memory database for development with Hibernate DDL auto-generation

### For Future Requirements & Analysis:

- **Scalability**: Current architecture supports horizontal scaling through stateless design
- **Maintainability**: Clear layer separation makes code maintainable and testable
- **Extensibility**: Service layer allows easy addition of new business logic
- **Security**: JWT tokens provide secure API access for frontend applications
- **Development**: H2 database enables rapid development and testing cycles

### Correlation with Your Requirements:

This architecture provides a solid foundation for:
- **API Development**: RESTful endpoints for all transport management operations
- **Data Management**: Comprehensive entity relationships for transport domain
- **Security**: Enterprise-grade authentication and authorization
- **Integration**: Easy frontend integration through CORS-enabled APIs
- **Future Analysis**: Well-structured data model supports analytics and reporting

## 9. DTO Schema Examples and Validation Patterns

This section demonstrates the Data Transfer Objects (DTOs) used in the application, showing request/response schemas, validation annotations, and how they integrate with the controller layer for data binding and validation.

In [ ]:
# DTO Schema Examples and Validation Architecture
def create_dto_schema_examples():
    schema_diagram = """
    ┌─────────────────────────────────────────────────────────────────────────────────────┐
    │                           DTO SCHEMA EXAMPLES & VALIDATION                         │
    │                        Request/Response Data Transfer Objects                       │
    └─────────────────────────────────────────────────────────────────────────────────────┘

    ┌─────────────────────────────────────────────────────────────────────────────────────┐
    │                              SignupRequest DTO                                     │
    └─────────────────────────────────────────────────────────────────────────────────────┘
    
    @NotBlank @Size(min=3, max=20)     @NotBlank @Size(max=50) @Email
    ┌─────────────────┐                ┌─────────────────┐
    │    username     │                │      email      │
    │   "john_doe"    │                │ "user@email.com"│
    └─────────────────┘                └─────────────────┘
    
    @NotBlank @Size(min=6, max=40)     @NotBlank @Size(max=50)
    ┌─────────────────┐                ┌─────────────────┐
    │    password     │                │   firstName     │
    │  "password123"  │                │     "John"      │
    └─────────────────┘                └─────────────────┘
    
    @NotBlank @Size(max=50)            @Size(max=15)           Set<String>
    ┌─────────────────┐                ┌─────────────────┐     ┌─────────────────┐
    │    lastName     │                │  phoneNumber    │     │      role       │
    │      "Doe"      │                │ "+1234567890"   │     │ ["USER","ADMIN"]│
    └─────────────────┘                └─────────────────┘     └─────────────────┘

    JSON Request Example:
    {
        "username": "john_doe",
        "email": "john.doe@example.com",
        "password": "securePassword123",
        "firstName": "John",
        "lastName": "Doe",
        "phoneNumber": "+1234567890",
        "role": ["USER"]
    }
    """
    
    print_banner("DTO SCHEMA EXAMPLES")
    print(schema_diagram)
    
    # More DTO examples
    additional_dtos = """
    ┌─────────────────────────────────────────────────────────────────────────────────────┐
    │                            Additional DTO Examples                                  │
    └─────────────────────────────────────────────────────────────────────────────────────┘
    
    DriverCreateRequest DTO:
    ┌─────────────────────────────────────────────────────────────────────────────────────┐
    │ @NotBlank String userId           | @NotBlank @Email String email                    │
    │ @NotBlank String firstName        | @NotBlank String phone                           │
    │ @NotBlank String lastName         | @NotNull LicenseType licenseType                │
    │ @Min(18) @Max(70) Integer age     | @NotNull Address address                        │
    │ @Min(0) Integer experience        | @Size(max=500) String specialization           │
    └─────────────────────────────────────────────────────────────────────────────────────┘
    
    VehicleRegistrationRequest DTO:
    ┌─────────────────────────────────────────────────────────────────────────────────────┐
    │ @NotBlank String vehicleNumber    | @NotBlank String ownerId                        │
    │ @NotNull VehicleType vehicleType  | @NotBlank String brand                          │
    │ @Min(1990) @Max(2025) Integer year| @NotBlank String model                          │
    │ @DecimalMin("0.1") BigDecimal cap | @NotNull DocumentInfo registration             │
    │ @NotNull InsuranceInfo insurance  | @NotNull Address location                      │
    └─────────────────────────────────────────────────────────────────────────────────────┘
    
    BookingRequest DTO:
    ┌─────────────────────────────────────────────────────────────────────────────────────┐
    │ @NotBlank String customerId       | @NotNull Address pickupAddress                 │
    │ @NotBlank String vehicleId        | @NotNull Address deliveryAddress               │
    │ @Future LocalDateTime scheduledTime| @DecimalMin("0") BigDecimal estimatedDistance │
    │ @Size(max=1000) String specialReq | @NotNull VehicleType preferredVehicleType     │
    └─────────────────────────────────────────────────────────────────────────────────────┘
    """
    
    print(additional_dtos)
    
    # Validation annotations reference
    validation_annotations = """
    ┌─────────────────────────────────────────────────────────────────────────────────────┐
    │                           VALIDATION ANNOTATIONS REFERENCE                          │
    └─────────────────────────────────────────────────────────────────────────────────────┘
    
    Basic Validations:
    ┌─────────────────────────────────────────────────────────────────┐
    │ @NotNull        - Field cannot be null                         │
    │ @NotBlank       - String cannot be null, empty, or whitespace  │
    │ @NotEmpty       - Collection/Array cannot be null or empty     │
    │ @Size(min, max) - String/Collection size constraints           │
    │ @Min(value)     - Numeric minimum value                        │
    │ @Max(value)     - Numeric maximum value                        │
    │ @Email          - Valid email format                           │
    │ @Pattern(regex) - Custom regex pattern matching                │
    └─────────────────────────────────────────────────────────────────┘
    
    Date/Time Validations:
    ┌─────────────────────────────────────────────────────────────────┐
    │ @Past           - Date must be in the past                     │
    │ @Future         - Date must be in the future                   │
    │ @PastOrPresent  - Date must be past or present                 │
    │ @FutureOrPresent- Date must be future or present               │
    └─────────────────────────────────────────────────────────────────┘
    
    Numeric Validations:
    ┌─────────────────────────────────────────────────────────────────┐
    │ @DecimalMin     - Decimal minimum value                        │
    │ @DecimalMax     - Decimal maximum value                        │
    │ @Positive       - Must be positive number                      │
    │ @PositiveOrZero - Must be positive or zero                     │
    │ @Negative       - Must be negative number                      │
    │ @NegativeOrZero - Must be negative or zero                     │
    └─────────────────────────────────────────────────────────────────┘
    """
    
    print(validation_annotations)

create_dto_schema_examples()

In [ ]:
# Controller Integration with DTO Validation
def create_controller_dto_integration():
    integration_diagram = """
    ┌─────────────────────────────────────────────────────────────────────────────────────┐
    │                    CONTROLLER-DTO INTEGRATION & VALIDATION FLOW                    │
    └─────────────────────────────────────────────────────────────────────────────────────┘
    
    Client Request Flow with DTO Validation:
    
    ┌─────────────────┐    ┌─────────────────┐    ┌─────────────────┐    ┌─────────────────┐
    │   Client JSON   │───▶│  Spring MVC     │───▶│ DTO Validation  │───▶│ Controller      │
    │   Request       │    │ Data Binding    │    │ @Valid          │    │ Method          │
    │                 │    │                 │    │                 │    │                 │
    │ {               │    │ @RequestBody    │    │ @NotBlank       │    │ @PostMapping    │
    │   "username":   │    │ SignupRequest   │    │ @Size           │    │ public Response │
    │   "john_doe",   │    │ signupRequest   │    │ @Email          │    │ signup(@Valid   │
    │   "email":      │    │                 │    │ Constraints     │    │ @RequestBody    │
    │   "john@.com",  │    │ Jackson         │    │                 │    │ SignupRequest)  │
    │   "password":   │    │ Deserialization │    │ Validation      │    │                 │
    │   "123"         │    │                 │    │ Errors          │    │                 │
    │ }               │    │                 │    │                 │    │                 │
    └─────────────────┘    └─────────────────┘    └─────────────────┘    └─────────────────┘
           │                        │                        │                        │
           │                        │                        │                        │
           ▼                        ▼                        ▼                        ▼
    JSON Payload            Object Creation         Validation Check         Business Logic
                           Field Mapping            Constraint Evaluation    Service Call
    
    Validation Error Response (422 Unprocessable Entity):
    {
        "timestamp": "2025-07-20T21:30:00.000+00:00",
        "status": 422,
        "error": "Validation Failed",
        "message": "Invalid input data",
        "path": "/api/auth/signup",
        "validationErrors": [
            {
                "field": "email",
                "rejectedValue": "john@.com",
                "message": "must be a well-formed email address"
            },
            {
                "field": "password",
                "rejectedValue": "123",
                "message": "size must be between 6 and 40"
            }
        ]
    }
    
    Success Response (201 Created):
    {
        "timestamp": "2025-07-20T21:30:00.000+00:00",
        "status": 201,
        "message": "User registered successfully",
        "data": {
            "userId": "uuid-12345",
            "username": "john_doe",
            "email": "john.doe@example.com",
            "firstName": "John",
            "lastName": "Doe",
            "roles": ["USER"]
        }
    }
    """
    
    print_banner("CONTROLLER-DTO INTEGRATION")
    print(integration_diagram)
    
    # Error handling patterns
    error_handling = """
    ┌─────────────────────────────────────────────────────────────────────────────────────┐
    │                           ERROR HANDLING PATTERNS                                   │
    └─────────────────────────────────────────────────────────────────────────────────────┘
    
    @ControllerAdvice Global Exception Handler:
    ┌─────────────────────────────────────────────────────────────────────────────────────┐
    │                                                                                     │
    │ @ExceptionHandler(MethodArgumentNotValidException.class)                           │
    │ public ResponseEntity<ErrorResponse> handleValidationExceptions(                   │
    │     MethodArgumentNotValidException ex) {                                          │
    │                                                                                     │
    │     List<ValidationError> errors = ex.getBindingResult()                          │
    │         .getFieldErrors()                                                          │
    │         .stream()                                                                  │
    │         .map(error -> new ValidationError(                                         │
    │             error.getField(),                                                      │
    │             error.getRejectedValue(),                                              │
    │             error.getDefaultMessage()))                                            │
    │         .collect(Collectors.toList());                                             │
    │                                                                                     │
    │     return ResponseEntity.status(HttpStatus.UNPROCESSABLE_ENTITY)                 │
    │         .body(new ErrorResponse("Validation Failed", errors));                    │
    │ }                                                                                   │
    │                                                                                     │
    └─────────────────────────────────────────────────────────────────────────────────────┘
    
    Custom Validation Annotations:
    ┌─────────────────────────────────────────────────────────────────────────────────────┐
    │ @Target({ElementType.FIELD})                                                       │
    │ @Retention(RetentionPolicy.RUNTIME)                                                │
    │ @Constraint(validatedBy = VehicleNumberValidator.class)                           │
    │ @Documented                                                                        │
    │ public @interface ValidVehicleNumber {                                             │
    │     String message() default "Invalid vehicle number format";                     │
    │     Class<?>[] groups() default {};                                                │
    │     Class<? extends Payload>[] payload() default {};                              │
    │ }                                                                                   │
    │                                                                                     │
    │ Usage in DTO:                                                                      │
    │ @ValidVehicleNumber                                                                │
    │ @NotBlank                                                                          │
    │ private String vehicleNumber; // Must match pattern: XX-00-XX-0000                │
    └─────────────────────────────────────────────────────────────────────────────────────┘
    """
    
    print(error_handling)

create_controller_dto_integration()

In [ ]:
# DTO Mapping and Data Flow Examples
def create_dto_mapping_examples():
    mapping_diagram = """
    ┌─────────────────────────────────────────────────────────────────────────────────────┐
    │                         DTO TO ENTITY MAPPING PATTERNS                             │
    └─────────────────────────────────────────────────────────────────────────────────────┘
    
    SignupRequest DTO → User Entity Conversion:
    
    ┌─────────────────────────┐                    ┌─────────────────────────┐
    │     SignupRequest       │     Service        │       User Entity       │
    │     (Input DTO)         │   Conversion       │    (Database Model)     │
    │                         │                    │                         │
    │ • username: "john_doe"  │ ──────────────────▶│ • id: UUID.generate()   │
    │ • email: "john@ex.com"  │                    │ • username: "john_doe"  │
    │ • password: "pass123"   │ passwordEncoder    │ • email: "john@ex.com"  │
    │ • firstName: "John"     │ .encode(password)  │ • password: "$2a$10..." │
    │ • lastName: "Doe"       │                    │ • firstName: "John"     │
    │ • phoneNumber: "+123"   │                    │ • lastName: "Doe"       │
    │ • role: ["USER"]        │ roleRepository     │ • phoneNumber: "+123"   │
    │                         │ .findByName("USER")│ • roles: [Role{USER}]   │
    │                         │                    │ • createdAt: LocalTime  │
    │                         │                    │ • updatedAt: LocalTime  │
    └─────────────────────────┘                    └─────────────────────────┘
    
    Driver Registration Flow:
    
    ┌─────────────────────────┐                    ┌─────────────────────────┐
    │   DriverCreateRequest   │     Service        │      Driver Entity      │
    │                         │   Mapping          │                         │
    │ • userId: "user123"     │ ──────────────────▶│ • id: UUID.generate()   │
    │ • firstName: "Alice"    │                    │ • userId: "user123"     │
    │ • lastName: "Smith"     │                    │ • firstName: "Alice"    │
    │ • email: "alice@ex.com" │                    │ • lastName: "Smith"     │
    │ • phone: "+1234567890"  │                    │ • email: "alice@ex.com" │
    │ • age: 28               │                    │ • phone: "+1234567890"  │
    │ • experience: 5         │                    │ • age: 28               │
    │ • licenseType: "CDL_A"  │ LicenseType        │ • experience: 5         │
    │ • address: {...}        │ .valueOf()         │ • licenseType: CDL_A    │
    │                         │                    │ • address: Address{}    │
    │                         │                    │ • status: PENDING       │
    │                         │                    │ • verificationStatus:   │
    │                         │                    │   NOT_VERIFIED          │
    │                         │                    │ • averageRating: 0.0    │
    │                         │                    │ • createdAt: now()      │
    └─────────────────────────┘                    └─────────────────────────┘
    """
    
    print_banner("DTO MAPPING PATTERNS")
    print(mapping_diagram)
    
    # Response DTO examples
    response_dtos = """
    ┌─────────────────────────────────────────────────────────────────────────────────────┐
    │                             RESPONSE DTO EXAMPLES                                   │
    └─────────────────────────────────────────────────────────────────────────────────────┘
    
    UserProfileResponse DTO:
    {
        "userId": "uuid-12345-67890",
        "username": "john_doe",
        "email": "john.doe@example.com",
        "firstName": "John",
        "lastName": "Doe",
        "phoneNumber": "+1234567890",
        "roles": ["USER", "DRIVER"],
        "accountStatus": "ACTIVE",
        "profileCompletion": 85,
        "lastLoginAt": "2025-07-20T21:30:00Z",
        "createdAt": "2025-01-15T10:00:00Z"
    }
    
    DriverDetailsResponse DTO:
    {
        "driverId": "driver-uuid-123",
        "personalInfo": {
            "userId": "user-uuid-456",
            "firstName": "Alice",
            "lastName": "Smith",
            "email": "alice.smith@example.com",
            "phone": "+1234567890",
            "age": 28
        },
        "professionalInfo": {
            "licenseType": "CDL_A",
            "totalYearsExperience": 5,
            "averageRating": 4.7,
            "totalRides": 347,
            "specialization": "Long Distance Transport"
        },
        "status": {
            "driverStatus": "AVAILABLE",
            "verificationStatus": "VERIFIED",
            "accountStatus": "ACTIVE"
        },
        "location": {
            "currentCity": "New York",
            "currentState": "NY",
            "latitude": 40.7128,
            "longitude": -74.0060,
            "lastUpdated": "2025-07-20T21:25:00Z"
        },
        "documents": {
            "licenseExpiry": "2026-12-31",
            "medicalCertExpiry": "2025-12-31",
            "backgroundCheckStatus": "CLEARED"
        }
    }
    
    VehicleRegistrationResponse DTO:
    {
        "vehicleId": "vehicle-uuid-789",
        "vehicleNumber": "NY-12-AB-3456",
        "ownerInfo": {
            "ownerId": "owner-uuid-101",
            "firstName": "Bob",
            "lastName": "Johnson",
            "contactEmail": "bob.johnson@example.com"
        },
        "vehicleDetails": {
            "vehicleType": "TRUCK",
            "brand": "Ford",
            "model": "F-150",
            "year": 2023,
            "capacity": 2.5,
            "fuelType": "DIESEL"
        },
        "registration": {
            "registrationNumber": "REG123456789",
            "registrationExpiry": "2026-06-30",
            "issuingAuthority": "NY DMV"
        },
        "insurance": {
            "policyNumber": "INS987654321",
            "provider": "State Farm",
            "policyExpiry": "2025-12-31",
            "coverageAmount": 500000.00
        },
        "status": {
            "vehicleStatus": "ACTIVE",
            "isVerified": true,
            "lastServiceDate": "2025-06-15",
            "nextServiceDue": "2025-12-15"
        },
        "location": {
            "currentCity": "Buffalo",
            "currentState": "NY",
            "parkingLocation": "Main Transport Hub"
        },
        "registrationTimestamp": "2025-07-20T15:30:00Z"
    }
    """
    
    print(response_dtos)
    
    # API endpoint examples
    api_examples = """
    ┌─────────────────────────────────────────────────────────────────────────────────────┐
    │                             API ENDPOINT EXAMPLES                                   │
    └─────────────────────────────────────────────────────────────────────────────────────┘
    
    Authentication Endpoints:
    ┌─────────────────────────────────────────────────────────────────────────────────────┐
    │ POST /api/auth/signup                                                               │
    │ Content-Type: application/json                                                      │
    │ Request Body: SignupRequest DTO                                                     │
    │ Response: 201 Created → UserRegistrationResponse DTO                               │
    │                                                                                     │
    │ POST /api/auth/signin                                                               │
    │ Content-Type: application/json                                                      │
    │ Request Body: LoginRequest DTO                                                      │
    │ Response: 200 OK → JwtAuthenticationResponse DTO                                   │
    │                                                                                     │
    │ POST /api/auth/refresh                                                              │
    │ Authorization: Bearer <refresh_token>                                               │
    │ Response: 200 OK → TokenRefreshResponse DTO                                        │
    └─────────────────────────────────────────────────────────────────────────────────────┘
    
    Driver Management Endpoints:
    ┌─────────────────────────────────────────────────────────────────────────────────────┐
    │ POST /api/drivers                                                                   │
    │ Authorization: Bearer <access_token>                                                │
    │ Content-Type: application/json                                                      │
    │ Request Body: DriverCreateRequest DTO                                               │
    │ Response: 201 Created → DriverDetailsResponse DTO                                  │
    │                                                                                     │
    │ GET /api/drivers/{driverId}                                                         │
    │ Authorization: Bearer <access_token>                                                │
    │ Response: 200 OK → DriverDetailsResponse DTO                                       │
    │                                                                                     │
    │ PUT /api/drivers/{driverId}                                                         │
    │ Authorization: Bearer <access_token>                                                │
    │ Content-Type: application/json                                                      │
    │ Request Body: DriverUpdateRequest DTO                                               │
    │ Response: 200 OK → DriverDetailsResponse DTO                                       │
    │                                                                                     │
    │ GET /api/drivers?status=AVAILABLE&city=NewYork&page=0&size=10                     │
    │ Authorization: Bearer <access_token>                                                │
    │ Response: 200 OK → PagedResponse<DriverSummaryResponse> DTO                        │
    └─────────────────────────────────────────────────────────────────────────────────────┘
    
    Vehicle Management Endpoints:
    ┌─────────────────────────────────────────────────────────────────────────────────────┐
    │ POST /api/vehicles                                                                  │
    │ Authorization: Bearer <access_token>                                                │
    │ Content-Type: application/json                                                      │
    │ Request Body: VehicleRegistrationRequest DTO                                        │
    │ Response: 201 Created → VehicleRegistrationResponse DTO                            │
    │                                                                                     │
    │ GET /api/vehicles/owner/{ownerId}                                                   │
    │ Authorization: Bearer <access_token>                                                │
    │ Response: 200 OK → List<VehicleSummaryResponse> DTO                                │
    │                                                                                     │
    │ PATCH /api/vehicles/{vehicleId}/status                                              │
    │ Authorization: Bearer <access_token>                                                │
    │ Content-Type: application/json                                                      │
    │ Request Body: VehicleStatusUpdateRequest DTO                                        │
    │ Response: 200 OK → VehicleStatusResponse DTO                                       │
    └─────────────────────────────────────────────────────────────────────────────────────┘
    """
    
    print(api_examples)

create_dto_mapping_examples()